In [ ]:
#START HERE#
#Open Libraries - Run every time
library(Seurat)
library(ggplot2)
library(dplyr)
library("EnhancedVolcano")
library(tictoc)
set.seed(42)

load("/Users/yzhou14/Data_local/20200627_scRNAseq_analysis/Colon_0.6_0730.RDS")
#after this step, an object named immune.combined is loaded
# we only choose the singlet for downstream analysis
immune.combined=subset(immune.combined,`DF`=="Singlet")
immune.combined[["celltype0627_stim"]]<- paste(Idents(immune.combined), immune.combined$stim, sep = "_")
immune.combined<-subset(immune.combined, 
                        celltype0627!="16.Epithelial cells" & 
                        celltype0627!="17.Endothelial cells"& 
                        celltype0627!="18.Fibroblasts")

In [ ]:
wti<-subset(immune.combined, stim=="WTI")
wtc<-subset(immune.combined, stim=="WTC")
koi<-subset(immune.combined, stim=="KOI")
koc<-subset(immune.combined, stim=="KOC")

In [ ]:
LRDB<-read.csv("LR_DASkelly.csv")

# WTI crosstalk

In [ ]:
seu<-wti
ligand.indata   <- unique(LRDB$ligand_symbol[LRDB$ligand_symbol %in%rownames(seu)])
receptor.indata <- unique(LRDB$receptor_symbol[LRDB$receptor_symbol %in%rownames(seu)])
lrpair.indata   <-(LRDB$ligand_symbol %in% ligand.indata)&(LRDB$receptor_symbol %in% receptor.indata)
LRDB            <- LRDB[lrpair.indata, ]
LRDB$ligand_symbol

In [ ]:
tic()
NPerm<-10000
Backgrounds<-list()
library(progress)
pb <- progress_bar$new(total = NPerm,format = "  generating [:bar] :percent in :elapsed eta: :eta",clear = T, width= 60,show_after=50)


for (i in 1:NPerm){
    pb$tick()
    #progress(i)
    # permute the cell identities
    identity<-seu$celltype0627
    names(identity)<-NULL
    set.seed(i)
    seu$random<-sample(identity)

    # compute the average expression after permutation
    Idents(seu)<-"random"
    expr <- AverageExpression(object = seu, assays = "RNA" ,slot = "data",verbose = F)$RNA
    expr[expr<expr.cutoff] <- 0
    expr <- as(as.matrix(expr), "dgCMatrix") 
    Backgrounds[[i]]<-list("expr.ligand"  =expr[LRDB$ligand_symbol,  ], 
                           "expr.receptor"=expr[LRDB$receptor_symbol,]) 
    if (i==NPerm) cat("Done!\n")
}
toc()

In [ ]:
save(Backgrounds, file="Crosstalk-wti.bkg.rda")

In [ ]:
expr.cutoff  <- 1.0
Idents(seu)<-"celltype0627"
expr <- AverageExpression(object = seu, assays = "RNA" ,slot = "data",verbose = F)$RNA
expr[expr<expr.cutoff] <- 0
expr <- as(as.matrix(expr), "dgCMatrix") 
expr.ligand  <-expr[LRDB$ligand_symbol,  ]
expr.receptor<-expr[LRDB$receptor_symbol,]


crosstalk<-list()
for (ct1 in levels(seu)){
    for (ct2 in levels(seu)){
        cat(ct1,"-> ⊃-",ct2,"\n")
        flush.console()
        # crosstalk between two cell types
        inner.prod <- expr.ligand[,ct1,drop=F] * expr.receptor[,ct2,drop=F]
        row.names(inner.prod) <- LRDB$pair
        LR.scores  <- sort(inner.prod[inner.prod[,1]>0,],decreasing = T)
        
        
        # compute p values
        LR.scores.pval<-rep(0,length(LR.scores))
        for(i in 1:NPerm){
            expr.ligand  <-Backgrounds[[i]]$expr.ligand
            expr.receptor<-Backgrounds[[i]]$expr.receptor
            # compute the background ligand-receptor interaction scores
            inner.prod <- expr.ligand[,ct1,drop=F] * expr.receptor[,ct2,drop=F]
            row.names(inner.prod) <- LRDB$pair
            LR.scores.null<-inner.prod[names(LR.scores),]
            LR.scores.pval<-LR.scores.pval+(as.numeric(LR.scores.null>LR.scores))
        }
        LR.scores.pval<-LR.scores.pval/NPerm
        
        
        crosstalk[[ct1]][[ct2]] <- data.frame(score=LR.scores, pval=LR.scores.pval)%>%arrange(pval)
    }
    
}

In [ ]:
pval.cutoff<-0.05
crosstalk.sig<-data.frame()

for (ct1 in levels(seu)){
    for (ct2 in levels(seu)){
        
        candidates<-crosstalk[[ct1]][[ct2]]
        candidates<-candidates[candidates$pval<pval.cutoff,]
        if (nrow(candidates)>0){
            candidates$ligand  <-ct1
            candidates$receptor<-ct2
            crosstalk.sig <- rbind(crosstalk.sig, candidates)
        }
        
        
    }
}
crosstalk.sig$score.log1p<-log1p(crosstalk.sig$score)
crosstalk.sig$LRpair<-rownames(crosstalk.sig)
write.csv(crosstalk.sig,file="Crosstalk-wti.sig.csv")
crosstalk.sig

# WTC crosstalk

In [ ]:
seu<-wtc
ligand.indata   <- unique(LRDB$ligand_symbol[LRDB$ligand_symbol %in%rownames(seu)])
receptor.indata <- unique(LRDB$receptor_symbol[LRDB$receptor_symbol %in%rownames(seu)])
lrpair.indata   <-(LRDB$ligand_symbol %in% ligand.indata)&(LRDB$receptor_symbol %in% receptor.indata)
LRDB            <- LRDB[lrpair.indata, ]
LRDB$ligand_symbol

In [ ]:
tic()
expr.cutoff  <- 1.0
NPerm<-10000
Backgrounds<-list()
library(progress)
pb <- progress_bar$new(total = NPerm,format = "  generating [:bar] :percent in :elapsed eta: :eta",clear = T, width= 60,show_after=50)


for (i in 1:NPerm){
    pb$tick()
    #progress(i)
    # permute the cell identities
    identity<-seu$celltype0627
    names(identity)<-NULL
    set.seed(i)
    seu$random<-sample(identity)

    # compute the average expression after permutation
    Idents(seu)<-"random"
    expr <- AverageExpression(object = seu, assays = "RNA" ,slot = "data",verbose = F)$RNA
    expr[expr<expr.cutoff] <- 0
    expr <- as(as.matrix(expr), "dgCMatrix") 
    Backgrounds[[i]]<-list("expr.ligand"  =expr[LRDB$ligand_symbol,  ], 
                           "expr.receptor"=expr[LRDB$receptor_symbol,]) 
    if (i==NPerm) cat("Done!\n")
}
toc()

In [ ]:
save(Backgrounds, file="Crosstalk-wtc.bkg.rda")

In [ ]:
expr.cutoff  <- 1.0
Idents(seu)<-"celltype0627"
expr <- AverageExpression(object = seu, assays = "RNA" ,slot = "data",verbose = F)$RNA
expr[expr<expr.cutoff] <- 0
expr <- as(as.matrix(expr), "dgCMatrix") 
expr.ligand  <-expr[LRDB$ligand_symbol,  ]
expr.receptor<-expr[LRDB$receptor_symbol,]


crosstalk<-list()
for (ct1 in levels(seu)){
    for (ct2 in levels(seu)){
        cat(ct1,"-> ⊃-",ct2,"\n")
        flush.console()
        # crosstalk between two cell types
        inner.prod <- expr.ligand[,ct1,drop=F] * expr.receptor[,ct2,drop=F]
        row.names(inner.prod) <- LRDB$pair
        LR.scores  <- sort(inner.prod[inner.prod[,1]>0,],decreasing = T)
        
        
        # compute p values
        LR.scores.pval<-rep(0,length(LR.scores))
        for(i in 1:NPerm){
            expr.ligand  <-Backgrounds[[i]]$expr.ligand
            expr.receptor<-Backgrounds[[i]]$expr.receptor
            # compute the background ligand-receptor interaction scores
            inner.prod <- expr.ligand[,ct1,drop=F] * expr.receptor[,ct2,drop=F]
            row.names(inner.prod) <- LRDB$pair
            LR.scores.null<-inner.prod[names(LR.scores),]
            LR.scores.pval<-LR.scores.pval+(as.numeric(LR.scores.null>LR.scores))
        }
        LR.scores.pval<-LR.scores.pval/NPerm
        
        
        crosstalk[[ct1]][[ct2]] <- data.frame(score=LR.scores, pval=LR.scores.pval)%>%arrange(pval)
    }
    
}

In [ ]:
pval.cutoff<-0.05
crosstalk.sig<-data.frame()

for (ct1 in levels(seu)){
    for (ct2 in levels(seu)){
        
        candidates<-crosstalk[[ct1]][[ct2]]
        candidates<-candidates[candidates$pval<pval.cutoff,]
        if (nrow(candidates)>0){
            candidates$ligand  <-ct1
            candidates$receptor<-ct2
            crosstalk.sig <- rbind(crosstalk.sig, candidates)
        }
        
        
    }
}
crosstalk.sig$score.log1p<-log1p(crosstalk.sig$score)
crosstalk.sig$LRpair<-rownames(crosstalk.sig)
write.csv(crosstalk.sig,file="Crosstalk-wtc.sig.csv")
crosstalk.sig

# KOI crosstalk

In [ ]:
seu<-koi
ligand.indata   <- unique(LRDB$ligand_symbol[LRDB$ligand_symbol %in%rownames(seu)])
receptor.indata <- unique(LRDB$receptor_symbol[LRDB$receptor_symbol %in%rownames(seu)])
lrpair.indata   <-(LRDB$ligand_symbol %in% ligand.indata)&(LRDB$receptor_symbol %in% receptor.indata)
LRDB            <- LRDB[lrpair.indata, ]
LRDB$ligand_symbol

In [ ]:
tic()
expr.cutoff  <- 1.0
NPerm<-10000
Backgrounds<-list()
library(progress)
pb <- progress_bar$new(total = NPerm,format = "  generating [:bar] :percent in :elapsed eta: :eta",clear = T, width= 60,show_after=50)


for (i in 1:NPerm){
    pb$tick()
    #progress(i)
    # permute the cell identities
    identity<-seu$celltype0627
    names(identity)<-NULL
    set.seed(i)
    seu$random<-sample(identity)

    # compute the average expression after permutation
    Idents(seu)<-"random"
    expr <- AverageExpression(object = seu, assays = "RNA" ,slot = "data",verbose = F)$RNA
    expr[expr<expr.cutoff] <- 0
    expr <- as(as.matrix(expr), "dgCMatrix") 
    Backgrounds[[i]]<-list("expr.ligand"  =expr[LRDB$ligand_symbol,  ], 
                           "expr.receptor"=expr[LRDB$receptor_symbol,]) 
    if (i==NPerm) cat("Done!\n")
}
toc()

In [ ]:
save(Backgrounds, file="Crosstalk-koi.bkg.rda")

In [ ]:
expr.cutoff  <- 1.0
Idents(seu)<-"celltype0627"
expr <- AverageExpression(object = seu, assays = "RNA" ,slot = "data",verbose = F)$RNA
expr[expr<expr.cutoff] <- 0
expr <- as(as.matrix(expr), "dgCMatrix") 
expr.ligand  <-expr[LRDB$ligand_symbol,  ]
expr.receptor<-expr[LRDB$receptor_symbol,]


crosstalk<-list()
for (ct1 in levels(seu)){
    for (ct2 in levels(seu)){
        cat(ct1,"-> ⊃-",ct2,"\n")
        flush.console()
        # crosstalk between two cell types
        inner.prod <- expr.ligand[,ct1,drop=F] * expr.receptor[,ct2,drop=F]
        row.names(inner.prod) <- LRDB$pair
        LR.scores  <- sort(inner.prod[inner.prod[,1]>0,],decreasing = T)
        
        
        # compute p values
        LR.scores.pval<-rep(0,length(LR.scores))
        for(i in 1:NPerm){
            expr.ligand  <-Backgrounds[[i]]$expr.ligand
            expr.receptor<-Backgrounds[[i]]$expr.receptor
            # compute the background ligand-receptor interaction scores
            inner.prod <- expr.ligand[,ct1,drop=F] * expr.receptor[,ct2,drop=F]
            row.names(inner.prod) <- LRDB$pair
            LR.scores.null<-inner.prod[names(LR.scores),]
            LR.scores.pval<-LR.scores.pval+(as.numeric(LR.scores.null>LR.scores))
        }
        LR.scores.pval<-LR.scores.pval/NPerm
        
        
        crosstalk[[ct1]][[ct2]] <- data.frame(score=LR.scores, pval=LR.scores.pval)%>%arrange(pval)
    }
    
}

In [ ]:
pval.cutoff<-0.05
crosstalk.sig<-data.frame()

for (ct1 in levels(seu)){
    for (ct2 in levels(seu)){
        
        candidates<-crosstalk[[ct1]][[ct2]]
        candidates<-candidates[candidates$pval<pval.cutoff,]
        if (nrow(candidates)>0){
            candidates$ligand  <-ct1
            candidates$receptor<-ct2
            crosstalk.sig <- rbind(crosstalk.sig, candidates)
        }
        
        
    }
}
crosstalk.sig$score.log1p<-log1p(crosstalk.sig$score)
crosstalk.sig$LRpair<-rownames(crosstalk.sig)
write.csv(crosstalk.sig,file="Crosstalk-koi.sig.csv")
crosstalk.sig

# KOC crosstalk

In [ ]:
seu<-koc
ligand.indata   <- unique(LRDB$ligand_symbol[LRDB$ligand_symbol %in%rownames(seu)])
receptor.indata <- unique(LRDB$receptor_symbol[LRDB$receptor_symbol %in%rownames(seu)])
lrpair.indata   <-(LRDB$ligand_symbol %in% ligand.indata)&(LRDB$receptor_symbol %in% receptor.indata)
LRDB            <- LRDB[lrpair.indata, ]
LRDB$ligand_symbol

In [ ]:
tic()
expr.cutoff  <- 1.0
NPerm<-10000
Backgrounds<-list()
library(progress)
pb <- progress_bar$new(total = NPerm,format = "  generating [:bar] :percent in :elapsed eta: :eta",clear = T, width= 60,show_after=50)


for (i in 1:NPerm){
    pb$tick()
    #progress(i)
    # permute the cell identities
    identity<-seu$celltype0627
    names(identity)<-NULL
    set.seed(i)
    seu$random<-sample(identity)

    # compute the average expression after permutation
    Idents(seu)<-"random"
    expr <- AverageExpression(object = seu, assays = "RNA" ,slot = "data",verbose = F)$RNA
    expr[expr<expr.cutoff] <- 0
    expr <- as(as.matrix(expr), "dgCMatrix") 
    Backgrounds[[i]]<-list("expr.ligand"  =expr[LRDB$ligand_symbol,  ], 
                           "expr.receptor"=expr[LRDB$receptor_symbol,]) 
    if (i==NPerm) cat("Done!\n")
}
toc()

In [ ]:
save(Backgrounds, file="Crosstalk-koc.bkg.rda")

In [ ]:
expr.cutoff  <- 1.0
Idents(seu)<-"celltype0627"
expr <- AverageExpression(object = seu, assays = "RNA" ,slot = "data",verbose = F)$RNA
expr[expr<expr.cutoff] <- 0
expr <- as(as.matrix(expr), "dgCMatrix") 
expr.ligand  <-expr[LRDB$ligand_symbol,  ]
expr.receptor<-expr[LRDB$receptor_symbol,]


crosstalk<-list()
for (ct1 in levels(seu)){
    for (ct2 in levels(seu)){
        cat(ct1,"-> ⊃-",ct2,"\n")
        flush.console()
        # crosstalk between two cell types
        inner.prod <- expr.ligand[,ct1,drop=F] * expr.receptor[,ct2,drop=F]
        row.names(inner.prod) <- LRDB$pair
        LR.scores  <- sort(inner.prod[inner.prod[,1]>0,],decreasing = T)
        
        
        # compute p values
        LR.scores.pval<-rep(0,length(LR.scores))
        for(i in 1:NPerm){
            expr.ligand  <-Backgrounds[[i]]$expr.ligand
            expr.receptor<-Backgrounds[[i]]$expr.receptor
            # compute the background ligand-receptor interaction scores
            inner.prod <- expr.ligand[,ct1,drop=F] * expr.receptor[,ct2,drop=F]
            row.names(inner.prod) <- LRDB$pair
            LR.scores.null<-inner.prod[names(LR.scores),]
            LR.scores.pval<-LR.scores.pval+(as.numeric(LR.scores.null>LR.scores))
        }
        LR.scores.pval<-LR.scores.pval/NPerm
        
        
        crosstalk[[ct1]][[ct2]] <- data.frame(score=LR.scores, pval=LR.scores.pval)%>%arrange(pval)
    }
    
}

In [ ]:
pval.cutoff<-0.05
crosstalk.sig<-data.frame()

for (ct1 in levels(seu)){
    for (ct2 in levels(seu)){
        
        candidates<-crosstalk[[ct1]][[ct2]]
        candidates<-candidates[candidates$pval<pval.cutoff,]
        if (nrow(candidates)>0){
            candidates$ligand  <-ct1
            candidates$receptor<-ct2
            crosstalk.sig <- rbind(crosstalk.sig, candidates)
        }
        
        
    }
}
crosstalk.sig$score.log1p<-log1p(crosstalk.sig$score)
crosstalk.sig$LRpair<-rownames(crosstalk.sig)
write.csv(crosstalk.sig,file="Crosstalk-koc.sig.csv")
crosstalk.sig

# Fig2 reproduction

In [ ]:
crosstalk.wti<-read.csv("Crosstalk-wti.sig.csv") %>% filter(pval<0.001)
crosstalk.wtc<-read.csv("Crosstalk-wtc.sig.csv") %>% filter(pval<0.001)
crosstalk.koi<-read.csv("Crosstalk-koi.sig.csv") %>% filter(pval<0.001)
crosstalk.koc<-read.csv("Crosstalk-koc.sig.csv") %>% filter(pval<0.001)

## Ligand set 1:'CCL2_CCR3','CCL4_CCR3','CCL5_CCR3'

In [ ]:
mycolors<-c("#98c1d9","#ff0054","#fdc500","#ff540090","#ff595e")
pair.all<-list(
          c('CCL2_CCR3','CCL4_CCR3','CCL5_CCR3'))

draw.crosstalk<-function(link_group, crosstalk.sig, link_cmap, pval.cutoff, link_width, lig_start=-1, rec_end=0.5){

    # get cell identity vector and rearrange them by cell types
    celltype.factor <- factor(
        as.character(immune.combined$celltype0627), 
        levels = c('0.IgD+ mature B','1.IgM+IgD+ mature B','11.IgD+ mature B','13.Proliferating B',
                   '6.IgA+ Plasma B','14.B plasmablast',
                   '2.CD4+CTLA4+ T', '3.CD4+ Teff','9.Foxp3+ Treg','10.CD8+ T, NKT',
                   '7.Gata3+ ILC2,ILC3','8.RORga+ ILC2,ILC3',
                   '4.Myeloid phagocytes','5.Ly2Z+ Myeloid','15.CX3CR1+ Mac','12.Neu')
    )
    celltype.df <- data.frame(
        cell_type=celltype.factor,
        x=runif(length(celltype.factor),-3,3),y=rnorm(length(celltype.factor))
    )

    library(circlize)
    # setting a proper color table and a cell type order
    colortable<-c('0.IgD+ mature B'= "#F5766D",
    '1.IgM+IgD+ mature B'="#E6812C",
    '2.CD4+CTLA4+ T'  ="#D38E00",
    '3.CD4+ Teff'  ="#B99A00",
    '4.Myeloid phagocytes'  ="#99A400",
    '5.Ly2Z+ Myeloid'  ="#6FAD00",
    '6.IgA+ Plasma B'  ="#00B513",
    '7.Gata3+ ILC2,ILC3'  ="#00BA61",
    '8.RORga+ ILC2,ILC3'  ="#00BD8B",
    '9.Foxp3+ Treg'  ="#00BDB1",
    '10.CD8+ T, NKT' ="#00BAD1",
    '11.IgD+ mature B' ="#00B2EB",
    '12.Neu' ="#00A4FF",
    '13.Proliferating B' ="#7F93FF",
    '14.B plasmablast' ="#B980FF",
    '15.CX3CR1+ Mac' ="#DF6EF4")

    order<-c('0.IgD+ mature B','1.IgM+IgD+ mature B','11.IgD+ mature B','13.Proliferating B',
                   '6.IgA+ Plasma B','14.B plasmablast',
                   '2.CD4+CTLA4+ T', '3.CD4+ Teff','9.Foxp3+ Treg','10.CD8+ T, NKT',
                   '7.Gata3+ ILC2,ILC3','8.RORga+ ILC2,ILC3',
                   '4.Myeloid phagocytes','5.Ly2Z+ Myeloid','15.CX3CR1+ Mac','12.Neu')
    colortable<-colortable[order]

    circos.par("track.height" = 0.1,points.overflow.warning=FALSE)
    circos.initialize(factors = celltype.df$cell_type, x = celltype.df$x)
    circos.track(factors = celltype.df$cell_type, y = celltype.df$y, bg.col=colortable)

    # add cell type labels
    for (ct in unique(immune.combined$celltype0627)){
        circos.text(0, 12, ct, sector.index = ct, track.index = 1,
                    niceFacing = T, cex=0.7,facing="bending.inside"
                   )
    }


    # add LR pair links
    lr.viz <- crosstalk.sig[crosstalk.sig$pval<pval.cutoff & crosstalk.sig$pair %in% link_group,] # change group here
    if (nrow(lr.viz)==0){
        return("empty\n")
    }
    
    start.pos<-list()
    for (start.ct in unique(lr.viz$ligand)){
        start.pos[[start.ct]]<-lig_start
    }
    
    end.pos<-list()
    for (end.ct in unique(lr.viz$receptor)){
        end.pos[[end.ct]]<-rec_end
    }
    
    for (row in 1:nrow(lr.viz)){
        start.ct<-lr.viz[row, "ligand"]
        end.ct<-lr.viz[row,"receptor"]
        
        circos.link(start.ct, start.pos[[start.ct]],
                    end.ct, end.pos[[end.ct]], 
                    h = 2.6, directional = T,
                    col=link_cmap[lr.viz[row, "pair"]], #change group here
                    lwd=link_width
                   )
        end.pos[[end.ct]]<-end.pos[[end.ct]]+0.17
        start.pos[[start.ct]]<-start.pos[[start.ct]]+0.55
    }
    circos.clear()
}

library(stringi)
for (pair in pair.all){
    pairs.to.show<-pair
    pairs.to.show.cmap<-mycolors[1:length(pairs.to.show)]
    names(pairs.to.show.cmap)<-pairs.to.show
    
    filename= stri_paste(pair, collapse=' ')
    filename=paste("./groupLRpair/",filename,".pdf",sep="")
    
    pdf(width =7*4, height = 6*1+3, file = filename)
    layout(matrix(1:4, 1, 4))
    draw.crosstalk(pairs.to.show,crosstalk.wti, pairs.to.show.cmap, pval.cutoff = 0.001, link_width =2.2 , lig_start = -2,rec_end = 0.8)
    title("WTI")
    draw.crosstalk(pairs.to.show,crosstalk.wtc, pairs.to.show.cmap, pval.cutoff = 0.001, link_width =2.2, lig_start = -2,rec_end = 0.8)
    title("WTC")
    draw.crosstalk(pairs.to.show,crosstalk.koi, pairs.to.show.cmap, pval.cutoff = 0.001, link_width =2.2, lig_start = -2,rec_end = 0.8 )
    title("KOI")
    draw.crosstalk(pairs.to.show,crosstalk.koc, pairs.to.show.cmap, pval.cutoff = 0.001, link_width =2.2, lig_start = -2,rec_end = 0.1)
    title("KOC")
    
    # add legends
    library(ComplexHeatmap)
    lgd_lines = Legend(at = names(pairs.to.show.cmap), type = "lines",  
                       legend_gp = gpar(col = pairs.to.show.cmap, lwd = 2),
                       title_position = "topleft",
                       title = "Ligand-receptor Pair")
    lgd_list_vertical = packLegend(lgd_lines)
    draw(lgd_list_vertical, x = unit(8, "mm"), y = unit(15, "mm"), just = c("left", "bottom"))

    dev.off()
    
    cat(filename, "\n")
}




## Ligand set 2: 'IFNG_IFNGR1','IFNG_IFNGR2','IL17A_IL17RA','IL10_IL10RA','IFNB1_IFNAR1','IFNB1_IFNAR2'

In [ ]:
mycolors<-c("#98c1d9","#8ac9268f","#fdc500","#ff540090","#7434819d","#ff0054")
pair.all<-list(
          c('IFNG_IFNGR1','IFNG_IFNGR2','IL17A_IL17RA','IL10_IL10RA','IFNB1_IFNAR1','IFNB1_IFNAR2'))

draw.crosstalk<-function(link_group, crosstalk.sig, link_cmap, pval.cutoff, link_width, lig_start=-1, rec_end=0.5){

    # get cell identity vector and rearrange them by cell types
    celltype.factor <- factor(
        as.character(immune.combined$celltype0627), 
        levels = c('0.IgD+ mature B','1.IgM+IgD+ mature B','11.IgD+ mature B','13.Proliferating B',
                   '6.IgA+ Plasma B','14.B plasmablast',
                   '2.CD4+CTLA4+ T', '3.CD4+ Teff','9.Foxp3+ Treg','10.CD8+ T, NKT',
                   '7.Gata3+ ILC2,ILC3','8.RORga+ ILC2,ILC3',
                   '4.Myeloid phagocytes','5.Ly2Z+ Myeloid','15.CX3CR1+ Mac','12.Neu')
    )
    celltype.df <- data.frame(
        cell_type=celltype.factor,
        x=runif(length(celltype.factor),-3,3),y=rnorm(length(celltype.factor))
    )

    library(circlize)
    # setting a proper color table and a cell type order
    colortable<-c('0.IgD+ mature B'= "#F5766D",
    '1.IgM+IgD+ mature B'="#E6812C",
    '2.CD4+CTLA4+ T'  ="#D38E00",
    '3.CD4+ Teff'  ="#B99A00",
    '4.Myeloid phagocytes'  ="#99A400",
    '5.Ly2Z+ Myeloid'  ="#6FAD00",
    '6.IgA+ Plasma B'  ="#00B513",
    '7.Gata3+ ILC2,ILC3'  ="#00BA61",
    '8.RORga+ ILC2,ILC3'  ="#00BD8B",
    '9.Foxp3+ Treg'  ="#00BDB1",
    '10.CD8+ T, NKT' ="#00BAD1",
    '11.IgD+ mature B' ="#00B2EB",
    '12.Neu' ="#00A4FF",
    '13.Proliferating B' ="#7F93FF",
    '14.B plasmablast' ="#B980FF",
    '15.CX3CR1+ Mac' ="#DF6EF4")

    order<-c('0.IgD+ mature B','1.IgM+IgD+ mature B','11.IgD+ mature B','13.Proliferating B',
                   '6.IgA+ Plasma B','14.B plasmablast',
                   '2.CD4+CTLA4+ T', '3.CD4+ Teff','9.Foxp3+ Treg','10.CD8+ T, NKT',
                   '7.Gata3+ ILC2,ILC3','8.RORga+ ILC2,ILC3',
                   '4.Myeloid phagocytes','5.Ly2Z+ Myeloid','15.CX3CR1+ Mac','12.Neu')
    colortable<-colortable[order]

    circos.par("track.height" = 0.1,points.overflow.warning=FALSE)
    circos.initialize(factors = celltype.df$cell_type, x = celltype.df$x)
    circos.track(factors = celltype.df$cell_type, y = celltype.df$y, bg.col=colortable)
    

    # add cell type labels
    for (ct in unique(immune.combined$celltype0627)){
        circos.text(0, 12, ct, sector.index = ct, track.index = 1,
                    niceFacing = T, cex=0.7,facing="bending.inside"
                   )
    }


    # add LR pair links
    lr.viz <- crosstalk.sig[crosstalk.sig$pval<pval.cutoff & crosstalk.sig$pair %in% link_group,] # change group here
    if (nrow(lr.viz)==0){
        return("empty\n")
    }
    
    start.pos<-list()
    for (start.ct in unique(lr.viz$ligand)){
        start.pos[[start.ct]]<-lig_start
    }
    
    end.pos<-list()
    for (end.ct in unique(lr.viz$receptor)){
        end.pos[[end.ct]]<-rec_end
    }
    
    for (row in 1:nrow(lr.viz)){
        start.ct<-lr.viz[row, "ligand"]
        end.ct<-lr.viz[row,"receptor"]
        
        circos.link(start.ct, start.pos[[start.ct]],
                    end.ct, end.pos[[end.ct]], 
                    h = 2.6, directional = T,
                    col=link_cmap[lr.viz[row, "pair"]], #change group here
                    lwd=link_width
                   )
        end.pos[[end.ct]]<-end.pos[[end.ct]]+0.07
        start.pos[[start.ct]]<-start.pos[[start.ct]]+0.15
    }
    circos.clear()
}

library(stringi)
for (pair in pair.all){
    pairs.to.show<-pair
    pairs.to.show.cmap<-mycolors[1:length(pairs.to.show)]
    names(pairs.to.show.cmap)<-pairs.to.show
    
    filename= stri_paste(pair, collapse=' ')
    filename=paste("./groupLRpair/",filename,".pdf",sep="")
    
    pdf(width =7*4, height = 6*1+3, file = filename)
    layout(matrix(1:4, 1, 4))
    draw.crosstalk(pairs.to.show,crosstalk.wti, pairs.to.show.cmap, pval.cutoff = 0.001, link_width =1.2 , lig_start = -2.5,rec_end = 0.8)
    title("WTI")
    draw.crosstalk(pairs.to.show,crosstalk.wtc, pairs.to.show.cmap, pval.cutoff = 0.001, link_width =1.2, lig_start = -2.5,rec_end = 0.8)
    title("WTC")
    draw.crosstalk(pairs.to.show,crosstalk.koi, pairs.to.show.cmap, pval.cutoff = 0.001, link_width =1.2, lig_start = -2.5,rec_end = 0.8 )
    title("KOI")
    draw.crosstalk(pairs.to.show,crosstalk.koc, pairs.to.show.cmap, pval.cutoff = 0.001, link_width =1.2, lig_start = -2.5,rec_end = 0.3)
    title("KOC")
    
    # add legends
    library(ComplexHeatmap)
    lgd_lines = Legend(at = names(pairs.to.show.cmap), type = "lines",  
                       legend_gp = gpar(col = pairs.to.show.cmap, lwd = 2),
                       title_position = "topleft",
                       title = "Ligand-receptor Pair")
    lgd_list_vertical = packLegend(lgd_lines)
    draw(lgd_list_vertical, x = unit(8, "mm"), y = unit(15, "mm"), just = c("left", "bottom"))

    dev.off()
    
    cat(filename, "\n")
}




## Ligand set 3: 'TNF_TNFRSF1A','TNF_TNFRSF1B','CXCL9_CCR3','CXCL10_CCR3','IL1A_IL1R2','IL1B_IL1R2'

In [ ]:
mycolors<-c("#98c1d9","#8ac9268f","#fdc500","#ff540090","#7434819d","#ff0054")
pair.all<-list(c('TNF_TNFRSF1A','TNF_TNFRSF1B','CXCL9_CCR3','CXCL10_CCR3','IL1A_IL1R2','IL1B_IL1R2'))

draw.crosstalk<-function(link_group, crosstalk.sig, link_cmap, pval.cutoff, link_width, lig_start=-1, rec_end=0.5){

    # get cell identity vector and rearrange them by cell types
    celltype.factor <- factor(
        as.character(immune.combined$celltype0627), 
        levels = c('0.IgD+ mature B','1.IgM+IgD+ mature B','11.IgD+ mature B','13.Proliferating B',
                   '6.IgA+ Plasma B','14.B plasmablast',
                   '2.CD4+CTLA4+ T', '3.CD4+ Teff','9.Foxp3+ Treg','10.CD8+ T, NKT',
                   '7.Gata3+ ILC2,ILC3','8.RORga+ ILC2,ILC3',
                   '4.Myeloid phagocytes','5.Ly2Z+ Myeloid','15.CX3CR1+ Mac','12.Neu')
    )
    celltype.df <- data.frame(
        cell_type=celltype.factor,
        x=runif(length(celltype.factor),-3,3),y=rnorm(length(celltype.factor))
    )

    library(circlize)
    # setting a proper color table and a cell type order
    colortable<-c('0.IgD+ mature B'= "#F5766D",
    '1.IgM+IgD+ mature B'="#E6812C",
    '2.CD4+CTLA4+ T'  ="#D38E00",
    '3.CD4+ Teff'  ="#B99A00",
    '4.Myeloid phagocytes'  ="#99A400",
    '5.Ly2Z+ Myeloid'  ="#6FAD00",
    '6.IgA+ Plasma B'  ="#00B513",
    '7.Gata3+ ILC2,ILC3'  ="#00BA61",
    '8.RORga+ ILC2,ILC3'  ="#00BD8B",
    '9.Foxp3+ Treg'  ="#00BDB1",
    '10.CD8+ T, NKT' ="#00BAD1",
    '11.IgD+ mature B' ="#00B2EB",
    '12.Neu' ="#00A4FF",
    '13.Proliferating B' ="#7F93FF",
    '14.B plasmablast' ="#B980FF",
    '15.CX3CR1+ Mac' ="#DF6EF4")

    order<-c('0.IgD+ mature B','1.IgM+IgD+ mature B','11.IgD+ mature B','13.Proliferating B',
                   '6.IgA+ Plasma B','14.B plasmablast',
                   '2.CD4+CTLA4+ T', '3.CD4+ Teff','9.Foxp3+ Treg','10.CD8+ T, NKT',
                   '7.Gata3+ ILC2,ILC3','8.RORga+ ILC2,ILC3',
                   '4.Myeloid phagocytes','5.Ly2Z+ Myeloid','15.CX3CR1+ Mac','12.Neu')
    colortable<-colortable[order]

    circos.par("track.height" = 0.1,points.overflow.warning=FALSE)
    circos.initialize(factors = celltype.df$cell_type, x = celltype.df$x)
    circos.track(factors = celltype.df$cell_type, y = celltype.df$y, bg.col=colortable)

    
    # add cell type labels
    for (ct in unique(immune.combined$celltype0627)){
        circos.text(0, 12, ct, sector.index = ct, track.index = 1,
                    niceFacing = T, cex=0.7,facing="bending.inside"
                   )
    }


    # add LR pair links
    lr.viz <- crosstalk.sig[crosstalk.sig$pval<pval.cutoff & crosstalk.sig$pair %in% link_group,] # change group here
    if (nrow(lr.viz)==0){
        return("empty\n")
    }
    
    start.pos<-list()
    for (start.ct in unique(lr.viz$ligand)){
        start.pos[[start.ct]]<-lig_start
    }
    
    end.pos<-list()
    for (end.ct in unique(lr.viz$receptor)){
        end.pos[[end.ct]]<-rec_end
    }
    
    for (row in 1:nrow(lr.viz)){
        start.ct<-lr.viz[row, "ligand"]
        end.ct<-lr.viz[row,"receptor"]
        
        circos.link(start.ct, start.pos[[start.ct]],
                    end.ct, end.pos[[end.ct]], 
                    h = 2.6, directional = T,
                    col=link_cmap[lr.viz[row, "pair"]], #change group here
                    lwd=link_width
                   )
        end.pos[[end.ct]]<-end.pos[[end.ct]]+0.07
        start.pos[[start.ct]]<-start.pos[[start.ct]]+0.15
    }
    circos.clear()
}

library(stringi)
for (pair in pair.all){
    pairs.to.show<-pair
    pairs.to.show.cmap<-mycolors[1:length(pairs.to.show)]
    names(pairs.to.show.cmap)<-pairs.to.show
    
    filename= stri_paste(pair, collapse=' ')
    filename=paste("./groupLRpair/",filename,".pdf",sep="")
    
    pdf(width =7*4, height = 6*1+3, file = filename)
    layout(matrix(1:4, 1, 4))
    draw.crosstalk(pairs.to.show,crosstalk.wti, pairs.to.show.cmap, pval.cutoff = 0.001, link_width =1.2 , lig_start = -1.5,rec_end = 0.8)
    title("WTI")
    draw.crosstalk(pairs.to.show,crosstalk.wtc, pairs.to.show.cmap, pval.cutoff = 0.001, link_width =1.2, lig_start = -1.5,rec_end = 0.8)
    title("WTC")
    draw.crosstalk(pairs.to.show,crosstalk.koi, pairs.to.show.cmap, pval.cutoff = 0.001, link_width =1.2, lig_start = -2.5,rec_end = 0.8 )
    title("KOI")
    draw.crosstalk(pairs.to.show,crosstalk.koc, pairs.to.show.cmap, pval.cutoff = 0.001, link_width =1.2, lig_start = -2.7,rec_end = 0.3)
    title("KOC")
    
    # add legends
    library(ComplexHeatmap)
    lgd_lines = Legend(at = names(pairs.to.show.cmap), type = "lines",  
                       legend_gp = gpar(col = pairs.to.show.cmap, lwd = 2),
                       title_position = "topleft",
                       title = "Ligand-receptor Pair")
    lgd_list_vertical = packLegend(lgd_lines)
    draw(lgd_list_vertical, x = unit(8, "mm"), y = unit(15, "mm"), just = c("left", "bottom"))

    dev.off()
    
    cat(filename, "\n")
}


